In [ ]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split

import optuna

from catboost import CatBoostClassifier

import sklearn.datasets
import sklearn.metrics

import os
import random
DEFAULT_RANDOM_SEED = 2021
def seedBasic(seed=DEFAULT_RANDOM_SEED):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    
seedBasic()

# global variabels

In [ ]:
TRAIN_PATH = "../input/tabular-playground-series-dec-2021/train.csv"
TEST_PATH = "../input/tabular-playground-series-dec-2021/test.csv"
SAMPLE_SUBMISSION_PATH = "../input/tabular-playground-series-dec-2021/sample_submission.csv"
SUBMISSION_PATH = "submission.csv"

ID = "Id"
TARGET = "Cover_Type"

N_TRIALS = 1
DIRECTIONS = "maximize"
TEST_SIZE = 0.25

# load data 

In [ ]:
train = pd.read_csv(TRAIN_PATH)
test = pd.read_csv(TEST_PATH)

In [ ]:
def reduce_memory_usage(df, verbose=True):
    numerics = ["int8", "int16", "int32", "int64", "float16", "float32", "float64"]
    start_mem = df.memory_usage().sum() / 1024 ** 2
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == "int":
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)
            else:
                if (
                    c_min > np.finfo(np.float16).min
                    and c_max < np.finfo(np.float16).max
                ):
                    df[col] = df[col].astype(np.float16)
                elif (
                    c_min > np.finfo(np.float32).min
                    and c_max < np.finfo(np.float32).max
                ):
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
    end_mem = df.memory_usage().sum() / 1024 ** 2
    if verbose:
        print(
            "Mem. usage decreased to {:.2f} Mb ({:.1f}% reduction)".format(
                end_mem, 100 * (start_mem - end_mem) / start_mem
            )
        )
    return df

train = reduce_memory_usage(train)
test = reduce_memory_usage(test)

# split data (input data and target data)

In [ ]:
y = train[TARGET]
X = train.drop([TARGET],axis=1)
X_test = test

# split data (train data and validataion data)

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=TEST_SIZE)

# make study and search best parameters

In [ ]:
def objective(trial):
   
    # 1.train
    param = {
        "iterations":trial.suggest_int("iterations", 500, 1000),
        "max_depth": trial.suggest_int("max_depth", 2, 7),
    }

    optunaModel = CatBoostClassifier(**param)
    optunaModel.fit(X_train,y_train,eval_set=[(X_val,y_val)],early_stopping_rounds=20,verbose=True)
    
    # 2.predict
    preds = optunaModel.predict(X_val)
    
    # 3.evaluate
    pred_labels = np.rint(preds)
    accuracy = sklearn.metrics.accuracy_score(y_val, pred_labels)
    
    return accuracy

study = optuna.create_study(direction=DIRECTIONS)
study.optimize(objective, n_trials=N_TRIALS)

# study result 

In [ ]:
study.best_trial

In [ ]:
study.best_trial.value

# best params

In [ ]:
study.best_trial.params

# build model using optuna best model

In [ ]:
model = CatBoostClassifier(**study.best_trial.params)

In [ ]:
model.fit(X_train,y_train)

# predict test data target 

In [ ]:
pred_test = model.predict(X_test)

# submission 

In [ ]:
sub = pd.read_csv(SAMPLE_SUBMISSION_PATH)
sub[TARGET] = pred_test
sub.to_csv(SUBMISSION_PATH,index=False)
sub.head()